## Import CDIP Sites

In [2]:
import pandas as pd
import geopandas
import numpy as np
import geopy
import geopy.distance
from shapely.geometry import Polygon


df = pd.read_csv('.../CDIP_Transects.csv')




In [3]:
distance_onshore=.1
distance_offshore=5


In [4]:
i=1 


df['upper_beach_bin_point']=np.nan
df['lower_beach_bin_point']=np.nan
df['upper_ocean_bin_point']=np.nan
df['lower_ocean_bin_point']=np.nan
df['site_bounds']=np.nan
for i in range(1, len(df)-1):
    pt_lat=df['Lat_BackBeach'].iloc[i]
    pt_lon=df['Lon_BackBeach'].iloc[i]
    beachpt_lat_upper=df['Lat_BackBeach'].iloc[i+1]
    beachpt_lon_upper=df['Lon_BackBeach'].iloc[i+1]
    beachpt_lat_lower=df['Lat_BackBeach'].iloc[i-1]
    beachpt_lon_lower=df['Lon_BackBeach'].iloc[i-1]


    pt_angle=df['ShoreNormal'].iloc[i]
    upper_angle=df['ShoreNormal'].iloc[i+1]
    lower_angle=df['ShoreNormal'].iloc[i-1]

    upper_bin_lat=(pt_lat+beachpt_lat_upper)/2
    upper_bin_lon=(pt_lon+beachpt_lon_upper)/2
    lower_bin_lat=(pt_lat+beachpt_lat_lower)/2
    lower_bin_lon=(pt_lon+beachpt_lon_lower)/2
    upper_normal=(pt_angle+upper_angle)/2
    lower_normal=(pt_angle+lower_angle)/2
    
    
    #upper beach bin point
    upper_start = geopy.Point(upper_bin_lat, upper_bin_lon)
    d = geopy.distance.distance(kilometers=(-1*distance_onshore))
    upper_beachbound = d.destination(point=upper_start, bearing=upper_angle).format_decimal()
    df['upper_beach_bin_point'].iloc[i]=upper_beachbound

    
    #lower beach bin point
    lower_start = geopy.Point(lower_bin_lat, lower_bin_lon)
    d = geopy.distance.distance(kilometers=(-1*distance_onshore))
    lower_beachbound = d.destination(point=lower_start, bearing=lower_angle).format_decimal()
    df['lower_beach_bin_point'].iloc[i]=lower_beachbound
    
    #upper ocean bin point
    upper_start = geopy.Point(upper_bin_lat, upper_bin_lon)
    d = geopy.distance.distance(kilometers=(distance_offshore))
    upper_oceanbound = d.destination(point=upper_start, bearing=upper_angle).format_decimal()
    df['upper_ocean_bin_point'].iloc[i]=upper_oceanbound
    
    #lower beach bin point
    lower_start = geopy.Point(lower_bin_lat, lower_bin_lon)
    d = geopy.distance.distance(kilometers=(distance_offshore))
    lower_oceanbound = d.destination(point=lower_start, bearing=lower_angle).format_decimal()
    df['lower_ocean_bin_point'].iloc[i]=lower_oceanbound
    
    

    lat_point_list = [float(lower_oceanbound.split(',')[0]), 
                      float(lower_beachbound.split(',')[0]), 
                      float(upper_beachbound.split(',')[0]), 
                      float(upper_oceanbound.split(',')[0])]
    lon_point_list = [float(lower_oceanbound.split(',')[1].split(' ')[1]),
                      float(lower_beachbound.split(',')[1].split(' ')[1]),
                      float(upper_beachbound.split(',')[1].split(' ')[1]), 
                      float(upper_oceanbound.split(',')[1].split(' ')[1])]
    

    polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
    df['site_bounds'].iloc[i]=polygon_geom
    #df['site_bounds'].iloc[i]=polygon_geom    
    




/home/williamspeiser/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


## Convert to GeoJson

In [20]:


from shapely import wkt

df['geometry'] = df['site_bounds'].astype(str)
df=df.dropna()
df['geometry']=df['geometry'].apply(wkt.loads)
df.to_csv('/home/williamspeiser/Desktop/CDIP_Binner/tst.csv')

df=pd.read_csv('/home/williamspeiser/Desktop/CDIP_Binner/tst.csv')
df['geometry']=df['geometry'].apply(wkt.loads)

df_geo = geopandas.GeoDataFrame(df, geometry = 'geometry',crs=4326)

df_geo.to_file('.../cdip_bins_surroundingshorenormals.geojson', driver='GeoJSON')